In [16]:
from pathlib import Path

inputdwi = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/dwi/sub-001_ses-01_acq-AxDTIASSET_dwi.nii"
sourcebvec = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/dwi/sub-001_ses-01_acq-AxDTIASSET_dwi.bvec"
sourcebval = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/dwi/sub-001_ses-01_acq-AxDTIASSET_dwi.bval"
fmap_rads = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/fmap/sub-001_ses-01_acq-RealFieldmapDTIrads_fmap.nii"
fmap_mag = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/fmap/sub-001_ses-01_acq-FieldmapDTI_magnitude1.nii"
outputdir = "/scratch/jdrussell3/fmaps"
dwelltime = 0.000568
diffusionTE = 62

def fmap_prep(inputdwi, fmap_rads, fmap_mag, dwelltime, diffusionTE):
    tmpdir = Path("/scratch/jdrussell3/b0corr")
    tmpdir.mkdir(exist_ok=True)
    
    #Copy phase and magnitude image to temp directory
    !fslmaths $fmap_rads $tmpdir/native_fmap_ph
    !fslmaths $fmap_mag $tmpdir/native_fmap_mag
    
    #Copy b0 image to temp directory, skull strip it, and save a binary mask file
    !fslroi $inputdwi $tmpdir/native_b0 0 7
    !fslmaths $tmpdir/native_b0 -Tmean $tmpdir/native_mnb0
    !bet2 $tmpdir/native_mnb0 $tmpdir/native_mnb0_brain -m
    
    #Skull strip magnitude image
    !bet2 $tmpdir/native_fmap_mag $tmpdir/native_fmap_mag_brain -m -f 0.3
    
    #Skull strip, dilate, and smooth the phase map
    !fslmaths $tmpdir/native_fmap_ph -mas $tmpdir/native_fmap_mag_brain_mask $tmpdir/native_fmap_ph_brain
    !fugue --verbose --loadfmap=$tmpdir/native_fmap_ph_brain -s 4 --savefmap=$tmpdir/native_fmap_ph_brain_smooth
    
    #Copy native_mnb0_brain, native_fmap_mag_brain, and native_fmap_ph_brain_smooth to processing folder
    #Delete temp folder

Reading fieldmap
Calculating masks
Calculating valid and filled masks
Regularising the fieldmap


In [ ]:
#Need source subject dir (in BIDSmaster), output subject dir, prefix

def dwi_corr(sourcedwi, sourcebvec, sourcebval, sourcefmap_rads, sourcefmap_mag, outputdwidir, eddy_options):
    #Get sub-XXX_ses-YY string
    prefix = "_".join(sourcedwi.name.split('_')[0:2])
                
    #Create directory structure
    origdir = Path(outputdwidir, 'original').mkdir(parents=True)
    preprocdir = Path(outputdwidir, 'preproc').mkdir(parents=True)
    
    #Copy source files to 'original' directory
    inputdwi = origdir /(prefix + "_dwi.nii")
    inputbvec = origdir /(prefix + "_dwi.bvec")
    inputbval = origdir /(prefix + "_dwi.bval")
    inputfmap_rads = origdir /(prefix + "_fmap_rads.nii")
    inputfmap_mag = origdir /(prefix + "_fmap_mag.nii")
    shutil.copyfile(sourcedwi, inputdwi)
    shutil.copyfile(sourcebvec, inputbvec)
    shutil.copyfile(sourcebval, inputbval)
    shutil.copyfile(sourcefmap_rads, inputfmap_rads)
    shutil.copyfile(sourcefmap_mag, inputfmap_mag)
       
    #################################
    #----Prepare fieldmap images----#
    #################################
    
    #1. Copy phase (in radians) and magnitude image to temp directory
    native_fmap_ph = $preprocdir/(prefix + "_native_fmap_ph.nii")
    native_fmap_mag = $preprocdir/(prefix + "_native_fmap_mag.nii")
    !fslmaths $inputfmap_rads $native_fmap_ph
    !fslmaths $inputfmap_mag $native_fmap_mag
    
    #2. Create a mean b0 image, then skull strip it, saving a binary mask file
    native_b0 = $preprocdir/(prefix + "_native_b0.nii")
    native_mnb0 = $preprocdir/(prefix + "_native_b0_brain.nii")
    native_mnb0_brain = $preprocdir/(prefix + "_native_mnb0_brain.nii.gz")
    native_mnb0_brain_mask = $preprocdir/(prefix + "_native_mnb0_brain_mask.nii.gz") 
    !fslroi $inputdwi $native_b0 0 7
    !fslmaths $native_b0 -Tmean $native_mnb0
    !bet2 $native_b0 $native_mnb0_brain -m
    
    #3. Skull strip the magnitude image and save a binary mask file
    native_fmap_mag_brain = $preprocdir/(prefix + "_native_fmap_mag_brain.nii.gz")
    native_fmap_mag_brain_mask = $preprocdir/(prefix + "_native_fmap_mag_brain_mask.nii.gz")
    !bet2 $native_fmap_mag $native_fmap_mag_brain -m -f 0.3
    
    #4. Brain mask the fieldmap using the binary mask of the magnitude image
    native_fmap_ph_brain = $preprocdir/(prefix + "_native_fmap_ph_brain.nii.gz")
    !fslmaths $tmpdir/native_fmap_ph -mas $tmpdir/native_fmap_mag_brain_mask $tmpdir/native_fmap_ph_brain
    
    #5. Smooth the field map
    native_fmap_ph_brain_s4 = $preprocdir/(prefix + "_native_fmap_ph_brain_s4.nii.gz")
    !fugue --verbose --loadfmap=$tmpdir/native_fmap_ph_brain -s 4 --savefmap=$tmpdir/native_fmap_ph_brain_s4

    exit(1)
    ############################################
    #----Removing Gibbs Rings and Denoising----#
    ############################################
    
    #1. Convert to MIF format
    dwi_raw = outdir / str(prefix + "_dwi.mif")
    !mrconvert -force -nthreads 12 -fslgrad $input_bvec $input_bval $input_dwi $dwi_raw
    
    #2. Denoise (MRTrix)
    dwi_den = outdir / str(prefix + "_dwi_den.mif")
    !dwidenoise -force $dwi_raw $dwi_den
    
    #3. Remove Gibbs rings
    dwi_den_deg = outdir / str(prefix + "_dwi_den_deg.mif")
    !mrdegibbs -force $dwi_den $dwi_den_deg
    
    #################################
    #----Eddy Current Correction----#
    #################################
    
    # Eddy Current Correction
    dwi_den_deg_preproc = outdir / str(prefix + "_dwi_den_deg_preproc.mif")
    outputdir_eddyqc = outdir / 'eddy_qc'
    !dwifslpreproc -force -info $dwi_den_deg $dwi_den_deg_preproc -pe_dir AP -rpe_none -scratch '/tmp' -eddy_options $eddy_options -eddyqc_all $outputdir_eddyqc
    
    ###################################
    #----EPI Distortion Correction----#
    ###################################
    #https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4819327/pdf/nihms767022.pdf
    
    ## Warp the magnitude image
    fugue -v -i native_fmap_mag_brain --unwarpdir=y --dwell=0.000568 --loadfmap= -w FieldMap/Magnitude_brain_warpped
    
    ## Register fmap_mag_brain_warp to nodif_brain and save the transformation matrix
    flirt -in FieldMap/Magnitude_brain_warped.nii.gz -ref DTI/nodif_brain.nii.gz -out FieldMap/Magnitude_brain_warpped_2_nodif_brain -omat FieldMap/fieldmap2diff.mat
    
    ## Register fmap_brain_s4 to the nodif_brain using the matrix obtained above
    flirt -in FieldMap/FieldMap_brain_s4.nii.gz -ref DTI/nodif_brain -applyxfm -init FieldMap/fieldmap2diff.mat -out FieldMap/FieldMap_brain_s4_2_nodif_brain
    
    ## Warp the corrected DTI using the newly registered fieldmap
    fugue -v -i DTI/data1_corr.nii.gz --icorr --unwarpdir=y --dwell=0.000720 --loadfmap=FieldMap/FieldMap_brain_s4_2_nodif_brain.nii.gz -u data/data.nii.gz
    
    ###############################################
    #----Bias Field (B1) Distortion Correction----#
    ###############################################
    
    #5. Bias correction
    dwi_den_deg_preproc_unb = outdir / str(prefix + "_dwi_den_deg_preproc_unb.mif")
    !dwibiascorrect -force ants $dwi_den_deg_preproc $dwi_den_deg_preproc_unb -scratch '/tmp'
    
    ######################################
    #----Upsampling and Mask Creation----#
    ######################################
    
    #6. Upsampling
    dwi_den_deg_preproc_unb_ups = outdir / str(prefix + "_dwi_den_deg_preproc_unb_ups.mif")
    !mrgrid $dwi_den_deg_preproc_unb regrid $dwi_den_deg_preproc_unb_ups -voxel 1.5
    #7. Mask generation
    ##a. Extract b0s
    dwi_den_deg_preproc_unb_b0s = outdir / str(prefix + "_dwi_den_deg_preproc_unb_b0s.mif")
    !dwiextract -force -bzero $dwi_den_deg_preproc_unb $dwi_den_deg_preproc_unb_b0s
    ##b. Compute mean b0
    dwi_den_deg_preproc_unb_meanb0 = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0.mif")
    !mrmath -force -axis 3 $dwi_den_deg_preproc_unb_b0s mean $dwi_den_deg_preproc_unb_meanb0
    ##c. Convert mean b0 to NII
    dwi_den_deg_preproc_unb_meanb0NII = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0.nii")
    !mrconvert -force $dwi_den_deg_preproc_unb_meanb0 $dwi_den_deg_preproc_unb_meanb0NII
    ##d. Create mask
    dwi_den_deg_preproc_unb_meanb0maskroot = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0")
    !bet2 $dwi_den_deg_preproc_unb_meanb0NII $dwi_den_deg_preproc_unb_meanb0maskroot -m
    ##e. Convert mask back to MIF
    dwi_den_deg_preproc_unb_meanb0maskNIIGZ = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0_mask.nii.gz")
    dwi_den_deg_preproc_unb_meanb0mask = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0_mask.mif")
    !mrconvert -force $dwi_den_deg_preproc_unb_meanb0maskNIIGZ $dwi_den_deg_preproc_unb_meanb0mask
    ##f. Upsample mask
    dwi_den_deg_preproc_unb_meanb0mask_ups = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0_mask_ups.mif")
    !mrgrid $dwi_den_deg_preproc_unb_meanb0mask regrid $dwi_den_deg_preproc_unb_meanb0mask_ups -template $dwi_den_deg_preproc_unb_ups -interp linear -datatype bit
    ##g. Filter mask
    dwi_den_deg_preproc_unb_meanb0mask_ups_filt = outdir / str(prefix + "_dwi_den_deg_preproc_unb_meanb0_mask_ups_filt.mif")
    !maskfilter $dwi_den_deg_preproc_unb_meanb0mask_ups median $dwi_den_deg_preproc_unb_meanb0mask_ups_filt
    

In [22]:
sourcedwi = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/dwi/sub-001_ses-01_acq-AxDTIASSET_dwi.nii"
sourcebvec = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/dwi/sub-001_ses-01_acq-AxDTIASSET_dwi.bvec"
sourcebval = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/dwi/sub-001_ses-01_acq-AxDTIASSET_dwi.bval"
sourcefmap = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/fmap/sub-001_ses-01_acq-RealFieldmapDTIrads_fmap.nii"
sourcemag = "/Volumes/Vol6/YouthPTSD/BIDS_master/sub-001/ses-01/fmap/sub-001_ses-01_acq-FieldmapDTI_magnitude1.nii"
outputdir = "/scratch/jdrussell3/fmaps"

input_dwi = "/scratch/jdrussell3/fmaps/original/sub-001_ses-01_dwi.nii"
input_bvec = "/scratch/jdrussell3/fmaps/original/sub-001_ses-01_dwi.nii"
input_bval = "/scratch/jdrussell3/fmaps/original/sub-001_ses-01_dwi.nii"
input_fmap = "/scratch/jdrussell3/fmaps/original/sub-001_ses-01_rads_fmap.nii"
input_mag = "/scratch/jdrussell3/fmaps/original/sub-001_ses-01_mag.nii"

dwi_corr(sourcedwi, sourcebvec, sourcebval, sourcefmap_rads, sourcefmap_mag, outputdwidir, eddy_options):

/scratch/jdrussell3/b0corr/sub-000_sex-99_dwi.nii
